In [1]:
import pandas as pd
import datetime
import numpy as np
import gzip
from sympy.solvers import solve
from sympy import Symbol

# Import Data

In [2]:
#import Reject Data
Reject_Loan_Data = {}
Reject_Loan_Data = pd.read_csv(r'''C:\Users\Charles\Desktop\Data Science\Capstone Project 2\rejected_2007_to_2018Q4.csv.gz''',dtype={19: str, 47: str, 55: str, 112: str, 123: str, 124: str, 125: str, 128: str, 129: str, 130: str, 133: str, 139: str, 140: str, 141: str})

In [3]:
#import Approved Data
Approved_Loan_Data = {}
Approved_Loan_Data = pd.read_csv(r'''C:\Users\Charles\Desktop\Data Science\Capstone Project 2\accepted_2007_to_2018Q4.csv.gz''',dtype={0: str,19: str, 49: str, 59: str, 118: str, 129: str, 130: str, 131: str, 134: str, 135: str, 136: str,139: str, 145: str, 146: str, 147: str})
#dtype={19: str, 47: str, 55: str, 112: str, 123: str, 124: str, 125: str, 128: str, 129: str, 130: str, 133: str, 139: str, 140: str, 141: str}

In [4]:
#truncate the columns of the approved data set in order to match the metrics of the rejected data set. 
Truncate_Columns = {}
Truncate_Columns = pd.read_csv(r'''C:\Users\Charles\Desktop\Data Science\Capstone Project 2\Truncate_Columns_v2.csv''')

# Clean up approved data

In [5]:
Truncate_Approved_Loan_Data = Approved_Loan_Data.filter(Truncate_Columns['Columns To Keep'])

In [6]:
# check for data with null data
#pd.options.display.max_seq_items = 2000
Truncate_Approved_Loan_Data.isnull().any()

emp_length                 True
addr_state                 True
dti                        True
dti_joint                  True
loan_amnt                  True
sec_app_fico_range_low     True
sec_app_fico_range_high    True
fico_range_high            True
fico_range_low             True
dtype: bool

In [7]:
from numpy import nan
null_data_delete = Truncate_Approved_Loan_Data[Truncate_Approved_Loan_Data['loan_amnt'].isnull()]
Clean_Truncate_Approved_Loan_Data=Truncate_Approved_Loan_Data.drop(null_data_delete.index.values)
null_data_delete = Clean_Truncate_Approved_Loan_Data[Clean_Truncate_Approved_Loan_Data['emp_length'].isnull()]
Clean_Truncate_Approved_Loan_Data=Clean_Truncate_Approved_Loan_Data.drop(null_data_delete.index.values)
null_data_delete = Clean_Truncate_Approved_Loan_Data[Clean_Truncate_Approved_Loan_Data['dti'].isnull()]
Clean_Truncate_Approved_Loan_Data=Clean_Truncate_Approved_Loan_Data.drop(null_data_delete.index.values)
Clean_Truncate_Approved_Loan_Data.isnull().any()

emp_length                 False
addr_state                 False
dti                        False
dti_joint                   True
loan_amnt                  False
sec_app_fico_range_low      True
sec_app_fico_range_high     True
fico_range_high            False
fico_range_low             False
dtype: bool

In [8]:
#average dti and dti_joint columns

Clean_Truncate_Approved_Loan_Data['Debt_To_Income_Ratio'] = Clean_Truncate_Approved_Loan_Data[['dti', 'dti_joint']].mean(axis=1)

In [9]:
#average primary and secondary borrowers' top and bottom range of FICO score. Approved data is all FICO score. Reject data has vantagescore starting in  2013
Clean_Truncate_Approved_Loan_Data['Risk_Score'] = Clean_Truncate_Approved_Loan_Data[['sec_app_fico_range_low', 'sec_app_fico_range_high','fico_range_high','fico_range_low']].mean(axis=1)

In [10]:
#Clean up column names
Clean_Truncate_Approved_Loan_Data = Clean_Truncate_Approved_Loan_Data.rename(columns={"emp_length": "Employment_Length", "addr_state": "State", "loan_amnt": "Loan_Amount"})
#Label data for approved: 1
Clean_Truncate_Approved_Loan_Data['Approved_Reject'] = 1

In [11]:
Clean_Truncate_Approved_Loan_Data.columns

Index(['Employment_Length', 'State', 'dti', 'dti_joint', 'Loan_Amount',
       'sec_app_fico_range_low', 'sec_app_fico_range_high', 'fico_range_high',
       'fico_range_low', 'Debt_To_Income_Ratio', 'Risk_Score',
       'Approved_Reject'],
      dtype='object')

In [12]:
Clean_Truncate_Approved_Loan_Data.isnull().any()

Employment_Length          False
State                      False
dti                        False
dti_joint                   True
Loan_Amount                False
sec_app_fico_range_low      True
sec_app_fico_range_high     True
fico_range_high            False
fico_range_low             False
Debt_To_Income_Ratio       False
Risk_Score                 False
Approved_Reject            False
dtype: bool

# Cleanup Reject Data

In [13]:
Reject_Loan_Data=Reject_Loan_Data[['Amount Requested','Application Date','Debt-To-Income Ratio', 'State','Employment Length', 'Risk_Score']]

In [14]:
# check for data with null data
Reject_Loan_Data.isnull().any()

Amount Requested        False
Application Date        False
Debt-To-Income Ratio    False
State                    True
Employment Length        True
Risk_Score               True
dtype: bool

In [15]:
null_data_delete = Reject_Loan_Data[Reject_Loan_Data['State'].isnull()]
Clean_Reject_Loan_Data=Reject_Loan_Data.drop(null_data_delete.index.values)
null_data_delete = Clean_Reject_Loan_Data[Clean_Reject_Loan_Data['Employment Length'].isnull()]
Clean_Reject_Loan_Data=Clean_Reject_Loan_Data.drop(null_data_delete.index.values)
null_data_delete = Clean_Reject_Loan_Data[Clean_Reject_Loan_Data['Risk_Score'].isnull()]
Clean_Reject_Loan_Data=Clean_Reject_Loan_Data.drop(null_data_delete.index.values)


In [16]:
Clean_Reject_Loan_Data.isnull().any()

Amount Requested        False
Application Date        False
Debt-To-Income Ratio    False
State                   False
Employment Length       False
Risk_Score              False
dtype: bool

In [17]:
#Clean up column names
Clean_Reject_Loan_Data = Clean_Reject_Loan_Data.rename(columns={"Employment Length": "Employment_Length", "State": "State", "Amount Requested": "Loan_Amount", "Debt-To-Income Ratio":"Debt_To_Income_Ratio"})
#Label data for approved: 1
Clean_Reject_Loan_Data['Approved_Reject'] = 0

# Reject Data Stratified Sampling

In [18]:
#Check Shape of Data Frame
R_r, R_c = Reject_Loan_Data.shape
CR_r, CR_c = Clean_Reject_Loan_Data.shape
TA_r, TA_c = Truncate_Approved_Loan_Data.shape
#list(Loan_Data)
print(R_r, R_c)
print(CR_r, CR_c)
print(TA_r,TA_c)
print(R_r/(TA_r+R_r))

27648741 6
8992576 7
2260701 9
0.9244151395402161


There are too many Reject Data relative to Approved data. We want to achieve 30/70 ratio

In [19]:
x = Symbol('x')
data_need = int(solve(TA_r/(TA_r+x)-.3, x)[0])
print("Need to reduce ", CR_r - data_need, " number of rows in reject data")

# % of reject data to retain. 
percentage_retain = data_need/CR_r

Need to reduce  3717607  number of rows in reject data


In [20]:
#convert from str to datetime
Clean_Reject_Loan_Data["Application Date"]=pd.to_datetime(Clean_Reject_Loan_Data["Application Date"], format = "%Y-%m-%d")

In [21]:
#Extract year from datetime
Clean_Reject_Loan_Data["Application_Date_Year"]=Clean_Reject_Loan_Data["Application Date"].dt.year

In [22]:
#stratified sampling based on % needed to be retained
Stratified_Clean_Reject_Loan_Data = Clean_Reject_Loan_Data.groupby('Application_Date_Year').apply(lambda x: x.sample(frac=percentage_retain))

In [23]:
Stratified_Clean_Reject_Loan_Data = Stratified_Clean_Reject_Loan_Data.reset_index(drop=True)

# Merge Approved and Rejected Data

In [24]:
Stratified_Clean_Reject_Loan_Data.columns

Index(['Loan_Amount', 'Application Date', 'Debt_To_Income_Ratio', 'State',
       'Employment_Length', 'Risk_Score', 'Approved_Reject',
       'Application_Date_Year'],
      dtype='object')

In [25]:
Clean_Truncate_Approved_Loan_Data.columns

Index(['Employment_Length', 'State', 'dti', 'dti_joint', 'Loan_Amount',
       'sec_app_fico_range_low', 'sec_app_fico_range_high', 'fico_range_high',
       'fico_range_low', 'Debt_To_Income_Ratio', 'Risk_Score',
       'Approved_Reject'],
      dtype='object')

In [26]:
Merged_Data = pd.concat([Stratified_Clean_Reject_Loan_Data[['Loan_Amount','Debt_To_Income_Ratio', 'State', 'Employment_Length','Risk_Score', 'Approved_Reject']], Clean_Truncate_Approved_Loan_Data[['Loan_Amount','Debt_To_Income_Ratio', 'State', 'Employment_Length','Risk_Score', 'Approved_Reject']]], axis = 0)

In [27]:
Merged_Data.columns

Index(['Loan_Amount', 'Debt_To_Income_Ratio', 'State', 'Employment_Length',
       'Risk_Score', 'Approved_Reject'],
      dtype='object')

# Split out States into Regions
## Based on Bureau of Economic Analysis regions split

In [28]:
#Check data quality of state column
Merged_Data.State.nunique()

51

In [29]:
test =Merged_Data.groupby('State').count()
test.loc['IA']

Loan_Amount             249
Debt_To_Income_Ratio    249
Employment_Length       249
Risk_Score              249
Approved_Reject         249
Name: IA, dtype: int64

In [36]:
#Merged_Data[Merged_Data['State']=='IA']

In [44]:
New_England = ['CT','ME', 'MA', 'NH', 'RI', 'VT']
Mideast = ['DE', 'DC', 'MD', 'NJ', 'NY', 'PA']
Great_Lakes = ['IL', 'ID','MI', 'OH','WI']
Plains = ['IA', 'KS', 'MN', 'MO', 'NE','ND','SD']
Southeast = ['AL', 'AR', 'FL', 'GA', 'KY','LA', 'MS', 'NC', 'SC', 'TN', 'VA', 'WV']
Southwest = ['AZ', 'NM', 'OK', 'TX']
Rocky_Mountain = ['CO','ID', 'MT', 'UT', 'WY']
Far_West = ['AK', 'CA', 'HI', 'NV', 'OR', 'WA']

In [54]:
def Search_List(List, data):
    if (data in List): 
        return True
    else:
        return False
    
Merged_Data['Region'] = Merged_Data['State'].apply(lambda x: 'New_England' if Search_List(New_England, x) else \
                                                   ('Mideast' if Search_List(Mideast, x) else \
                                                   ('Great_Lakes' if Search_List(Great_Lakes, x) else \
                                                   ('Plains' if Search_List(Plains, x) else \
                                                   ('Southwest' if Search_List(Southwest, x) else \
                                                   ('Rock_Mountain' if Search_List(Rocky_Mountain, x) else \
                                                   ('Far_West' if Search_List(Far_West, x) else 'N/A')))))))

In [55]:
Merged_Data[Merged_Data['Region']=='Far_West']

,Loan_Amount,Debt_To_Income_Ratio,State,Employment_Length,Risk_Score,Approved_Reject,Region
5,2500.0,4.66%,CA,1 year,524.0,0,Far_West
22,1000.0,8.27%,WA,< 1 year,553.0,0,Far_West
29,25000.0,24.22%,WA,< 1 year,650.0,0,Far_West
31,25000.0,0.55%,WA,10+ years,544.0,0,Far_West
52,8000.0,22.22%,CA,4 years,619.0,0,Far_West
61,1500.0,8.22%,WA,< 1 year,469.0,0,Far_West
79,5000.0,210.59%,CA,3 years,476.0,0,Far_West
151,7000.0,23.63%,CA,9 years,628.0,0,Far_West
152,500.0,0%,WA,2 years,591.0,0,Far_West
194,3000.0,0%,AK,10+ years,0.0,0,Far_West


# Data Exploration

In [31]:
pd.options.display.max_seq_items = 2000
Approved_Loan_Data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'last_fico_range_high', 'last_fico_range_low',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_co

In [32]:
#Check Shape of Data Frame
R_r, R_c = Reject_Loan_Data.shape
CR_r, CR_c = Clean_Reject_Loan_Data.shape
SCR_r, SCR_c = Stratified_Clean_Reject_Loan_Data.shape
TA_r, TA_c = Truncate_Approved_Loan_Data.shape

#list(Loan_Data)
print(R_r, R_c)
print(CR_r, CR_c)
print(SCR_r, SCR_c)
print(TA_r,TA_c)

27648741 6
8992576 8
5274969 8
2260701 9


In [33]:
Clean_Reject_Loan_Data.groupby('Application_Date_Year').count()

,Loan_Amount,Application Date,Debt_To_Income_Ratio,State,Employment_Length,Risk_Score,Approved_Reject
Application_Date_Year,,,,,,,
2007,5170,5170,5170,5170,5170,5170,5170
2008,23070,23070,23070,23070,23070,23070,23070
2009,50752,50752,50752,50752,50752,50752,50752
2010,102088,102088,102088,102088,102088,102088,102088
2011,213098,213098,213098,213098,213098,213098,213098
2012,329356,329356,329356,329356,329356,329356,329356
2013,726041,726041,726041,726041,726041,726041,726041
2014,1648116,1648116,1648116,1648116,1648116,1648116,1648116
2015,498001,498001,498001,498001,498001,498001,498001


In [34]:
Stratified_Clean_Reject_Loan_Data.groupby('Application_Date_Year').count()

,Loan_Amount,Application Date,Debt_To_Income_Ratio,State,Employment_Length,Risk_Score,Approved_Reject
Application_Date_Year,,,,,,,
2007,3033,3033,3033,3033,3033,3033,3033
2008,13533,13533,13533,13533,13533,13533,13533
2009,29771,29771,29771,29771,29771,29771,29771
2010,59884,59884,59884,59884,59884,59884,59884
2011,125001,125001,125001,125001,125001,125001,125001
2012,193197,193197,193197,193197,193197,193197,193197
2013,425890,425890,425890,425890,425890,425890,425890
2014,966771,966771,966771,966771,966771,966771,966771
2015,292123,292123,292123,292123,292123,292123,292123


In [35]:
Stratified_Clean_Reject_Loan_Data

,Loan_Amount,Application Date,Debt_To_Income_Ratio,State,Employment_Length,Risk_Score,Approved_Reject,Application_Date_Year
0,12000.0,2007-11-30,28.62%,KS,< 1 year,770.0,0,2007
1,4000.0,2007-10-09,221.94%,FL,1 year,605.0,0,2007
2,4000.0,2007-12-28,5.86%,PA,5 years,641.0,0,2007
3,5000.0,2007-08-10,6.16%,GA,1 year,591.0,0,2007
4,1500.0,2007-07-19,2.91%,WI,< 1 year,529.0,0,2007
5,2500.0,2007-12-21,4.66%,CA,1 year,524.0,0,2007
6,3500.0,2007-12-10,19.86%,VA,1 year,507.0,0,2007
7,8000.0,2007-12-29,2.31%,MD,3 years,571.0,0,2007
8,15000.0,2007-11-29,19.1%,MA,6 years,621.0,0,2007
9,5000.0,2007-12-31,10.83%,IL,1 year,458.0,0,2007
